<a href="https://colab.research.google.com/github/smkh1985/SC_valuechain/blob/main/Value_Chain_GRAPH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.0/171.0 KB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/d3/b7/f1f8ffdb12d7e10d31b0a23bc2d8a78ad4659474eda2a62f023717df2ba1/neo4j-5.5.0.tar.gz#sha256=2632386380b2ebb7d6a80e4186899ef342ef0507601c65e200696f13742046b8 (from https://pypi.org/simple/neo4j/) (requires-python:>=3.7): Requested unknown from https://files.pythonhosted.org/packages/d3/b7/f1f8ffdb12d7e10d31b0a23bc2d8a78ad4659474eda2a62f023717df2ba1/neo4j-5.5.0.tar.gz#sha256=2632386380b2ebb7d6a80e4186899ef342ef0507601c65e200696f13742046b8 has inconsistent name: filename has 'neo4j', but metadata has 'unknown'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 KB 17.1 MB/s eta 0:00

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable


import pandas as pd

import xlrd


In [16]:
df = pd.read_excel("ValueChains.xlsx",skiprows=5)
df.columns = [name.replace(" ","_") for name in df.columns]
df['Last_Update_Date'] = pd.to_datetime(df['Last_Update_Date']).apply(lambda x: x.date())

In [8]:

workbook = xlrd.open_workbook('ValueChains.xlsx')
worksheet = workbook.sheet_by_name('Value Chains')
main_company_name = worksheet.cell(2, 1).value
main_company_id =  worksheet.cell(3, 1).value

In [9]:
class ValueChainGraphDB():
  def __init__(self , uri , username , password):
    self.driver = None
    try :
      self.driver = GraphDatabase.driver(uri , auth= (username,password))
    except Exception as e:
            print("Failed to create the driver:", e)


  def close(self):
    self.driver.close()

  
  def query(self, query, parameters=None, db=None):
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.driver.session(database=db) if db is not None else self.driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

  def create_constraint(self):
    self.query("CREATE CONSTRAIN unique_company ON (company:Company) ASSERT company.name IS UNIQUE")


In [10]:
graph_db = ValueChainGraphDB("neo4j+s://12d6e328.databases.neo4j.io","neo4j","d--ukQGquG-o2T4gQGaArAK4cZLGtjBoBDhM-yJqe28")

In [46]:
def add_companies(companies):
    # Adds category nodes to the Neo4j graph.
    query = '''
            UNWIND $rows AS row
            MERGE (c:Company {name: row.Company_Name, id:row.Identifier})
            RETURN count(*) as total
            '''
    return graph_db.query(query, parameters = {'rows':companies})


def add_relationship(main_id,rls):
    # Adds category nodes to the Neo4j graph.
    query = '''
            MATCH (c1:Company {id:$main_id})
            UNWIND $rows AS row
            MATCH (c2:Company {id:row.Identifier})
            CREATE (c1)-[:LINK]->(c2)
            RETURN count(*) as total
            '''
    return graph_db.query(query, parameters = {'rows':rls , 'main_id':main_id})


In [47]:
add_companies(df.loc[:,['Identifier','Company_Name']].to_dict('records'))
add_companies({'Identifier':main_company_id,'Company_Name':main_company_name})

[<Record total=1>]

In [48]:
add_relationship(main_company_id,df.to_dict('records'))

[<Record total=356>]